<a href="https://colab.research.google.com/github/omkarsk98/NewsDataClassification/blob/development/NewsDataClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-10-01 10:46:02--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.176.181
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.176.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

        GoogleNews-  28%[====>               ] 442.47M  33.9MB/s    eta 36s    ^C


In [0]:
!git clone https://github.com/omkarsk98/NewsDataClassification.git

Cloning into 'NewsDataClassification'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 5), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [0]:
%cd NewsDataClassification

/content/NewsDataClassification


In [0]:
!git checkout development

Branch 'development' set up to track remote branch 'development' from 'origin'.
Switched to a new branch 'development'


In [0]:
!gunzip GoogleNews-vectors-negative300.bin.gz

In [0]:
!ls

GoogleNews-vectors-negative300.bin  NewsDataClassification.ipynb  README.md
index.py			    News_Final.csv


In [0]:
import pandas as pd
from gensim import models
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [0]:
# read csv
main_data = pd.read_csv('News_Final.csv')
# read titles from it
article_titles = main_data['TITLE']
# Create a list of strings, one for each title
titles_list = [title for title in article_titles]
# form a single string fro the list of strings
big_title_string = ' '.join(titles_list)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# Tokenize the string into words
tokens = word_tokenize(big_title_string)

# Remove non-alphabetic tokens, such as punctuation
words = [word.lower() for word in tokens if word.isalpha()]
stop_words = set(stopwords.words('english'))
words = [word for word in words if not word in stop_words]

In [0]:
# Load word2vec model (trained on an enormous Google corpus)
model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True) 
# Check dimension of word vectors
print("Dimensions of the model",model.vector_size)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Dimensions of the model 300


In [0]:
# Filter the list of vectors to include only those that Word2Vec has a vector for
vector_list = [model[word] for word in words if word in model.vocab]
# Create a list of the words corresponding to these vectors
words_filtered = [word for word in words if word in model.vocab]

# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)

# Cast to a dict so we can turn it into a DataFrame
word_vec_dict = dict(word_vec_zip)
df = pd.DataFrame.from_dict(word_vec_dict, orient='index')

# save dataframe to csv just for reference
df.to_csv('Words_vs_Vectors.csv')

In [0]:
from google.colab import files
files.download('Words_vs_Vectors.csv')

In [0]:
print("Number of words found:", len(df))

Number of words found: 35133


In [0]:
# Using some pre built functions
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in model.vocab]
    return np.mean(model[doc], axis=0)

# Our earlier preprocessing was done when we were dealing only with word vectors
# Here, we need each document to remain a document 
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] 
    return doc

# Function that will help us drop documents that have no word vectors in word2vec
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

# Filter out documents
def filter_docs(corpus, texts, condition_on_doc):
    """
    Filter corpus and texts given the function condition_on_doc which takes a doc. The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]

    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)

In [1]:
# # Preprocess the corpus to get list of documents with stop words removed and containing only the words that are present in the vocab
# corpus = [preprocess(title) for title in titles_list]
# # corpus[0] => ['fed', 'official', 'says', 'weak', 'data', 'caused', 'weather', 'slow', 'taper']
# # corpus[1] => ['fed', 'charles', 'plosser', 'sees', 'high', 'bar', 'change', 'pace', 'tapering']

# # Remove docs that don't include any words in W2V's vocab
# corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: has_vector_representation(model, doc))
# # corpus[0] => ['fed', 'official', 'says', 'weak', 'data', 'caused', 'weather', 'slow', 'taper']
# # corpus[1] => ['fed', 'charles', 'plosser', 'sees', 'high', 'bar', 'change', 'pace', 'tapering']
# # titles_list[0] => Fed official says weak data caused by weather, should not slow taper
# # titles_list[1] => Fed's Charles Plosser sees high bar for change in pace of tapering


# # Filter out any empty docs
# corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: (len(doc) != 0))
# # corpus[0] => ['fed', 'official', 'says', 'weak', 'data', 'caused', 'weather', 'slow', 'taper']
# # titles_list => Fed official says weak data caused by weather, should not slow taper

x = []
for doc in corpus: # append the vector for each document
    x.append(document_vector(model, doc))
    
vectorsForEachDocument = np.array(x) # list to array
len(vectorsForEachDocument)

NameError: ignored

In [0]:
# drop all rows that dont have category
main_data = main_data.drop(main_data[main_data['CATEGORY'].isna()].index)
main_data.shape

(422398, 13)

**What all does the data contain at the moment.**

---
corpus: A list containing a list of words of which stop words are removed. Only the words that are present in the vocab are present here.<br>
titles_list: It contains a list of all titles.<br>
vectorsForEachDocument: It contains vectors for each document.<br>

In [0]:
from sklearn.manifold import TSNE

# Initialize t-SNE
tsne = TSNE(n_components = 2, init = 'random', random_state = 10, perplexity = 100)

# Use only 400 rows to shorten processing time
tsne_df = tsne.fit_transform(df[:400])
tsne_df